## A/B testing
- формулировка гипотез
- проверка нормальности
- Mann–Whitney
- выводы

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import stats, shapiro, mannwhitneyu

In [4]:
# чтение данных
test_group = pd.read_csv('test_group.csv', sep=';')
control_group = pd.read_csv('control_group.csv', sep=';')
# переименование колонок для удобства
test_group.rename(columns={'# of Impressions': 'Impressions of ad', '# of Website Clicks': 'Website Clicks', '# of Searches': 'Searches', '# of View Content': 'Viewed content', '# of Add to Cart': 'Added to Cart', '# of Purchase': 'Purchases'}, inplace=True)
control_group.rename(columns={'# of Impressions': 'Impressions of ad', '# of Website Clicks': 'Website Clicks', '# of Searches': 'Searches', '# of View Content': 'Viewed content', '# of Add to Cart': 'Added to Cart', '# of Purchase': 'Purchases'}, inplace=True)
# из-за пропусков в одной строчке данных в control_group убираем ее из датафрейма

control_group['CVR'] = control_group['Purchases'].apply(lambda x: x * 100) / control_group['Impressions of ad']
test_group['CVR'] = test_group['Purchases'].apply(lambda x: x * 100) / test_group['Impressions of ad']
test_group['CPA'] = test_group['Spend [USD]'] / test_group['Purchases']
control_group['CPA'] = control_group['Spend [USD]'] / control_group['Purchases']
test_group['CTR'] = test_group['Website Clicks'] * 100 / test_group['Impressions of ad']
control_group['CTR'] = control_group['Website Clicks'] * 100 / control_group['Impressions of ad']
subset_without_1_line = control_group.dropna(subset=['Impressions of ad', 'Website Clicks', 'Purchases'])

df = pd.concat([subset_without_1_line, test_group], ignore_index=True)

In [5]:
# проверим на нормальное распределение CVR и CPA
print(shapiro(df[df['Campaign Name']=='Control Campaign']['CVR']))
print(shapiro(df[df['Campaign Name']=='Test Campaign']['CVR']))
print(shapiro(df[df['Campaign Name']=='Control Campaign']['CPA']))
print(shapiro(df[df['Campaign Name']=='Test Campaign']['CPA']))

ShapiroResult(statistic=0.9573368197700145, pvalue=0.28217276932475077)
ShapiroResult(statistic=0.8838476002711686, pvalue=0.0034594715637168527)
ShapiroResult(statistic=0.9131872183920033, pvalue=0.02056352984195968)
ShapiroResult(statistic=0.8965798196078477, pvalue=0.006934224858783165)


## Выбор теста
Исходя из теста Шапиро-Уилка делаем вывод, что данные преимущественно распределены ненормально.
Также у нас достаточно маленькая выборка данных.
Исходя из этих соображений будем использовать тест Манна-Уитни.

## Гипотезы
Будем исследовать CPA и CVR

    Для CVR:
    - H0: control(CVR) = test(CVR)
    - H1: control(CVR) < test(CVR)

    Для CPA:
    - H0: control(CPA) = test(CPA)
    - H1: control(CPA) > test(CPA)

In [6]:
control_cvr = df[df['Campaign Name'] == 'Control Campaign']['CVR']
test_cvr = df[df['Campaign Name'] == 'Test Campaign']['CVR']

u, p = mannwhitneyu(test_cvr, control_cvr, alternative='greater')
print("CVR Mann–Whitney p-value:", p)

CVR Mann–Whitney p-value: 0.00203173807001684


In [7]:
control_cpa = df[df['Campaign Name'] == 'Control Campaign']['CPA']
test_cpa = df[df['Campaign Name'] == 'Test Campaign']['CPA']

u, p = mannwhitneyu(test_cpa, control_cpa, alternative='less')
print("CPA Mann–Whitney p-value:", p)

CPA Mann–Whitney p-value: 0.8674470725536065


## Промежуточные выводы
1. p-value для CVR существенно меньше 0.05; из этого следует, что в тестовой стратегии наблюдается значительный рост этого показателя.
2. p-value для CPA очень близок к 1, можно предположить, что CPA у тестовой стратегии не меньше, и даже не равен, а, наоборот, больше чем у контрольной стратегии.

Проведем еще один тест для CPA. Теперь Н1: control(CPA) < test(CPA)

In [8]:
u, p = mannwhitneyu(test_cpa, control_cpa, alternative='greater')
print("CPA Mann–Whitney p-value:", p)

CPA Mann–Whitney p-value: 0.13583122616258847


Дополнительный тест не дает 100% уверенности в том, что CPA будет всегда больше при тестовой стратегии. Воспользуемся bootstrap для более точной оценки.